In [ ]:
import hyperspy.api as hs
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import pandas as pd

from helper_functions import elementlines, nearestlines, theoretical_energy

In [ ]:
import os
path = "../../Masteroppgave/2023-03-08_EDS-Apreo/exports/"
files = os.listdir(path)
files

In [ ]:
s_list = [hs.load(path + '/' + file, signal_type='EDS_SEM') for file in files]

In [ ]:
zero_peak_slice = 33 # zero peak slice in calibration of GaAs from Sept 2022
scale = 0.01
offset = 0.1242 - zero_peak_slice * scale
channels = 2048
x = np.arange(offset, offset + channels * scale, scale)

In [ ]:
def figg(log=True):
    fig = go.Figure()
    fig.update_layout(margin=dict(l=5, r=5, t=25, b=5))
    fig.update_layout(xaxis_title='Energy (keV)', yaxis_title='Counts')
    if log:
        fig.update_yaxes(type="log")
    return fig


def add_all_element_lines(fig, element):
    lines = list(hs.material.elements[element].Atomic_properties.Xray_lines.as_dictionary().keys())
    for line in lines:
        line = element + '_' + line
        fig.add_vline(x=theoretical_energy(line), line_width=1, line_dash="dash", line_color="black", annotation_text=line, annotation_position="top right", annotation_textangle=90)
    
    
def kev_to_channel(energy):
    return int((energy - offset) / scale)

def add_line(fig, line, color='grey'):
    fig.add_vline(x=theoretical_energy(line), line_width=0.5, line_dash="dash", line_color=color, annotation_text=line, annotation_position="top right", annotation_textangle=90)

def add_multiple_lines(fig, lines, color='grey'):
    for line in lines:
        add_line(fig, line, color)

In [ ]:
elementlines('Sb')
# Sb is missing the Mz line, previously mistaken for the N_Ka line

In [ ]:
# plot raw data
fig = figg(log=False)
for s in s_list:
    fig.add_trace(go.Scatter(x=x, y=s.data, name=s.metadata.General.original_filename))

# add_all_element_lines(fig, 'Ga')
# add_all_element_lines(fig, 'O')
# add_all_element_lines(fig, 'C')
# add_all_element_lines(fig, 'N')
add_all_element_lines(fig, 'Sb')
# add_all_element_lines(fig, 'Si')
# add_all_element_lines(fig, 'As')
# add_all_element_lines(fig, 'Cu')

fig.show()

In [ ]:
# plot all from March 2023
fig = figg()
for s in s_list:
    fig.add_trace(go.Scatter(x=x, y=s.data, name=s.metadata.General.original_filename))

fig.update_xaxes(range=[0.1, 11])
fig.update_yaxes(range=[0.5, 5.5])

# add_multiple_lines(fig,['Ga_Ka', 'Ga_La', 'Ga_Kb', 'Ga_Ll', 'C_Ka', 'Si_Ka', 'Si_Kb', 'N_Ka', 'O_Ka'], 'black')
# add_multiple_lines(fig,['Sb_La', 'Sb_Ll', 'Sb_Lg1', 'Sb_Lg3', 'Sb_Lb1', 'Sb_Lb3', 'Sb_Lb2'], 'grey')

# add_all_element_lines(fig, 'Sb')

# fig.write_html("figures/all_March2023.html")
# fig.show()
'f'

In [ ]:
# plot GaSb
fig = figg()
for s in s_list:
    fname = s.metadata.General.original_filename
    if 'GaSb' in fname:
        if 'processTime' in fname:
            continue
        if '00pA' in fname or 'scratched' in fname or 'noPPUC' in fname:
            fig.add_trace(go.Scatter(x=x, y=s.data, name=fname, visible='legendonly'))

        else:
            fig.add_trace(go.Scatter(x=x, y=s.data, name=fname))

fig.update_xaxes(range=[0.1, 11])
fig.update_yaxes(range=[0.5, 5.5])

add_multiple_lines(fig,['Ga_Ka', 'Ga_La', 'Ga_Kb', 'Ga_Ll', 'C_Ka', 'Si_Ka', 'Si_Kb', 'O_Ka'], 'black')
add_multiple_lines(fig,['Sb_La', 'Sb_Ll', 'Sb_Lg1', 'Sb_Lg3', 'Sb_Lb1', 'Sb_Lb3', 'Sb_Lb4', 'Sb_Lb2'], 'grey')

# annotate Sb_Mz at x=0.42 with a vline
fig.add_vline(x=0.42, line_width=0.5, line_dash="dash", line_color='grey', annotation_text='Sb_Mz? (in AZtec, not in HS)', annotation_position="top right", annotation_textangle=90)
fig.add_vline(x=0.73, line_width=0.5, line_dash="dash", line_color='grey', annotation_text='Sb_Mg? (in AZtec, not in HS)', annotation_position="top right", annotation_textangle=90)


# add_all_element_lines(fig, 'Sb')

# fig.write_html("figures/GaSb_March2023.html")
# fig.show()
'f'

In [ ]:
# plot GaSb, just different process times
fig = figg()
for s in s_list:
    fname = s.metadata.General.original_filename
    if 'GaSb' in fname:
        if 'processTime' in fname and '50pA' in fname:
            fig.add_trace(go.Scatter(x=x, y=s.data, name=fname))

for s in s_list:
    fname=s.metadata.General.original_filename
    if 'GaSb' in fname:
        if '400pA' in fname:
            fig.add_trace(go.Scatter(x=x, y=s.data, name=fname, visible='legendonly'))


fig.update_xaxes(range=[0.1, 11])
fig.update_yaxes(range=[2.2, 5.5])

add_multiple_lines(fig,['Ga_Ka', 'Ga_La', 'Ga_Kb', 'Ga_Ll', 'C_Ka', 'Si_Ka', 'Si_Kb', 'N_Ka', 'O_Ka'], 'black')
add_multiple_lines(fig,['Sb_La', 'Sb_Ll', 'Sb_Lg1', 'Sb_Lg3', 'Sb_Lb1', 'Sb_Lb3', 'Sb_Lb2'], 'grey')
fig.add_annotation(x=7.42, y=4.2, text="Sb sum peaks on purple line", showarrow=False, font=dict(size=10, color="gray"))

# fig.write_html("figures/GaSb_ProcessTimes_March2023.html")
# fig.show()
'f'

In [ ]:
# plot GaAs
fig = figg()
for s in s_list:
    if 'GaAs' in s.metadata.General.original_filename:
        if 'Cu' in s.metadata.General.original_filename:
            fig.add_trace(go.Scatter(x=x, y=s.data, name=s.metadata.General.original_filename, visible='legendonly'))
        else:
            fig.add_trace(go.Scatter(x=x, y=s.data, name=s.metadata.General.original_filename))

fig.update_xaxes(range=[0.1, 12.2])
fig.update_yaxes(range=[0.5, 5.5])



add_multiple_lines(fig,['Ga_Ka', 'Ga_La', 'Ga_Kb', 'Ga_Ll', 'As_Ka', 'As_La', 'As_Kb', 'C_Ka', 'Si_Ka'], 'black')
add_multiple_lines(fig, [ 'Ag_La', 'Ag_Lb1', 'Cu_Ka', 'Cu_Kb', 'Cu_La'], 'grey')


# fig.write_html("figures/GaAs_March2023.html")

In [ ]:
# showing the effect of different processTime

processTimeFiles = [
# 'GaSb_30kV_50pA_noPPUC.emsa', #  'PT 6 (no PPUC)',
'GaSb_30kV_50pA_processTime1.emsa',
'GaSb_30kV_50pA_processTime2.emsa',
'GaSb_30kV_50pA_processTime4.emsa',
'GaSb_30kV_50pA.emsa']

names = ['Process time 1', 'Process time 2', 'Process time 4', 'Process time 6']

processTimeData = [hs.load(path + '/' + file, signal_type='EDS_SEM').data for file in processTimeFiles]

fig = figg()
for i, s in enumerate(processTimeData):
    fig.add_trace(go.Scatter(x=x, y=s, name=names[i]))

fig.update_xaxes(range=[0.17, 1.3])
fig.update_yaxes(range=[2.3, 4.4])
fig.update_layout(title_text='Effect of different process times, all lines are GaSb 30kV 50pA (DT and real time is different)')
fig.update_layout(margin=dict(l=5, r=5, t=50, b=5))
fig.update_layout(legend=dict( yanchor="top", y=0.93, xanchor="left", x=0.01))
# fig.update_layout(width=1000, height=600)
add_multiple_lines(fig,['Ga_La', 'Ga_Ll', 'C_Ka', 'O_Ka', 'C_Ka', 'N_Ka'], 'black')

'p'

fig.show()

fig.write_html("figures/processTime.html")